# Crack identification procedure

In [ ]:
%matplotlib widget
import ibvpy.api as ib
import matplotlib.pylab as plt
from matplotlib import cm
from bmcs_shear.dic_crack import\
    DICGrid, DICInpUnstructuredPoints, DICStateFields, \
    DICAlignedGrid, DICGrid
from bmcs_shear.dic_crack.dic_crack_list2 import DICCrackList
import numpy as np
np.seterr(divide ='ignore', invalid='ignore');

In [ ]:
dic_points = DICInpUnstructuredPoints(U_factor=10, dir_name='B9_TV2', t=1,
                                      n_T_max=40, T_stepping='delta_T')
dic_points.read_beam_design()
dic_grid = DICGrid(dic_inp=dic_points, d_x=5, d_y=5)

In [ ]:
dsf = DICStateFields(dic_grid=dic_grid, R=6, omega_threshold=0.15, verbose_eval=True)
dsf.tmodel_.trait_set(E=1400, c_T=0.006, nu=0.18, epsilon_0=0.0018,
                      epsilon_f=0.0019, eps_max=0.01);
dsf = DICStateFields(dic_grid=dic_grid, R=5, omega_threshold=0.15, verbose_eval=True, tmodel='scalar_damage')
dsf.tmodel_.trait_set(E=1600, nu=0.18, omega_fn='exp-slope', strain_norm='Rankine', eps_max=0.01)
dsf.tmodel_.omega_fn_.trait_set(kappa_0=0.002, kappa_f=0.0028);
dcl = DICCrackList(dsf=dsf, delta_alpha_min=-np.pi/6, delta_alpha_max=np.pi/3, delta_s = 25, x_boundary=30)

In [ ]:
dcl.interact()

In [ ]:
%%capture

from matplotlib import animation, rc
from IPython.display import HTML
from traitlets import HasTraits
import matplotlib.gridspec as gridspec


fig = plt.figure(figsize=(12,3))

In [ ]:
class CrackListAnimator(HasTraits):
    
    
    def init(self):
        print('init')
        n_T = dsf.dic_grid.n_T
        self.t_dic_T = np.hstack([
            np.linspace(0,1,n_T),
            np.ones((int(0.5*n_T),))
        ])
    
    def subplots(self, fig):
        gs = gridspec.GridSpec(ncols=2, nrows=1,
                               width_ratios=[3, 1],
                               #wspace=0.5,
                               hspace=0.5,
                               # height_ratios=[2, 1]
                               )
        ax_dcl = fig.add_subplot(gs[0, 0])
        ax_FU = fig.add_subplot(gs[0, 1])
#        return fig.subplots(1,1)
#        return ax_dsf#, ax_FU
        return ax_dcl, ax_FU

    def plot(self, i):
        fig.clear()
        t = self.t_dic_T[i]
        print('t', t)
        axes = self.subplots(fig)
        dcl.dsf.dic_grid.t = t
        
        ax_dcl, ax_FU = axes
#        ax_dsf = axes
#        ax_FU = axes
        
        dcl.bd.plot_sz_bd(ax_dcl)
        dcl.dsf.plot_crack_detection_field(ax_dcl, fig)
        dcl.plot_cracking_hist2(ax_dcl)
        dcl.critical_crack.plot_x_t_crc_Ka(ax_dcl, line_width=2, line_color='red', tip_color='red')
        ax_dcl.axis('equal')
        ax_dcl.axis('off');
        dcl.dsf.dic_grid.plot_load_deflection(ax_FU)
        
    def mp4_video(self):
        n_t = len(self.t_dic_T)
        # call the animator. blit=True means only re-draw the parts that have changed.
        anim = animation.FuncAnimation(fig, self.plot, init_func=self.init,
                                       frames=n_t, interval=300, blit=True)
        return anim.save("cracking_animation.gif")
    
    def html5_video(self):
        n_t = len(self.t_dic_T)
        # call the animator. blit=True means only re-draw the parts that have changed.
        anim = animation.FuncAnimation(fig, self.plot, init_func=self.init,
                                       frames=n_t, interval=300, blit=True)
        return anim.to_html5_video()

In [ ]:
# anim = CrackListAnimator()
# anim.init()
# anim.mp4_video()

In [ ]:
anim = CrackListAnimator()
anim.init()
with open('cracking_animation.html','w') as html_video_file:
    html_video_file.write(anim.html5_video())

In [ ]:
html_video_file = open('cracking_animation.html','r')
HTML(html_video_file.read())

Given the step length $\Delta s$ get the damage values in the distance $s$ along an angle $\alpha_\min, \alpha_\max$

In [ ]:
from scipy.signal import argrelextrema
self=dcl
n_G = 40
###
delta_s = 10
delta_alpha_left = np.pi / 6
delta_alpha_right = np.pi / 6

In [ ]:
# spatial coordinates
xx_MN, yy_MN, omega_irn_1_MN = self.dsf.omega_irn_1_MN
# number of points to skip on the left and right side based on the x_boundary parameters
d_x = xx_MN[1,0] - xx_MN[0,0]
M_offset = int(self.x_boundary / d_x)
# initial crack positions at the bottom of the zone
M_C_with_offset = argrelextrema(omega_irn_1_MN[M_offset:-M_offset, 0], np.greater)[0]
M_C = M_C_with_offset + M_offset
# running and stopped cracks
n_C = len(M_C)
C_r = np.arange(n_C)
# initial points
x_C0, y_C0 = xx_MN[M_C, 0], yy_MN[M_C, 0]
X_C0a = np.array([x_C0, y_C0]).T
X_C0a.shape

In [ ]:
def get_X_C1a(X_C0a, C_r, alpha_C0):
    # active crack tips     
    X_r0a = X_C0a[C_r]
    alpha_r0a = alpha_C0[C_r]
    # range of angles in each crack
    alpha_min_r1 = alpha_r0a - delta_alpha_left
    alpha_max_r1 = alpha_r0a + delta_alpha_right
    alpha_gr1 = np.linspace(alpha_min_r1, alpha_max_r1, n_G)
    alpha_r1g = alpha_gr1.T
    # range of points around the crack tip
    delta_X_agr1 = np.array([-np.sin(alpha_gr1), np.cos(alpha_gr1)]) * delta_s
    delta_X_r1ga = np.einsum('agr->rga', delta_X_agr1)
    # global position of candidate crack tips
    X_r1ga = X_r0a[:, np.newaxis, :] + delta_X_r1ga
    x_r1g, y_r1g = np.einsum('...a->a...', X_r1ga)
    # damage values in candidate crack tips
    t_r1g = np.ones_like(x_r1g)
    args = (t_r1g, x_r1g, y_r1g)
    omega_r1g = self.dsf.f_omega_irn_txy(args)
    # index of candidate with maximum damage in each active tip 
    arg_g_omega_r1 = np.argmax(omega_r1g, axis=-1)
    r_r = np.arange(len(arg_g_omega_r1))
    max_omega_r1 = omega_r1g[r_r, arg_g_omega_r1]
    alpha_r1 = alpha_r1g[r_r, arg_g_omega_r1]
    # Update active crack tips
    C_C = np.arange(len(X_C0a))
    r_running = np.where(max_omega_r1 > 0.1)
    C_r = C_C[C_r[r_running]]
    # new crack tip
    X_r1a = X_r1ga[r_r, arg_g_omega_r1]
    X_C1a = np.copy(X_C0a)
    X_C1a[C_r] = X_r1a[r_running]
    alpha_C1 = np.copy(alpha_C0)
    alpha_C1[C_r] = alpha_r1[r_running]
    return X_C1a, C_r, alpha_C1

In [ ]:
X_KCa_ = [X_C0a]
alpha_C0 = np.zeros((len(X_C0a),))
while len(C_r) > 0:
    X_C1a, C_r, alpha_C0 = get_X_C1a(X_C0a, C_r, alpha_C0)
    X_KCa_.append(X_C1a)
    X_C0a = X_C1a
X_KCa = np.array(X_KCa_);

In [ ]:
%matplotlib widget
fig, ax = plt.subplots(1,1,figsize=(8,2.5))
T = -1
x_MN, y_MN = np.einsum('...a->a...', dsf.X_ipl_MNa)
contour_levels = np.linspace(0,1,10)
ax.contourf( x_MN, y_MN, dsf.omega_irn_TMN[T], contour_levels, cmap='BuPu')

X_aCK = np.einsum('KCa->aKC', X_KCa)
ax.plot(*X_aCK, 'o-', color='orange');
ax.axis('equal');
ax.axis('off');